In [2]:
from keras.applications import VGG16
pretrained_model = VGG16(include_top=True, weights='imagenet')
pretrained_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [30]:
from keras.preprocessing.image import ImageDataGenerator

# create a new generator
imagegen = ImageDataGenerator()
# load train data
batch_size = 1
train = imagegen.flow_from_directory("imagenette2/train/", class_mode="categorical", shuffle=True, batch_size=batch_size, target_size=(224, 224))
# load val data
val = imagegen.flow_from_directory("imagenette2/val/", class_mode="categorical", shuffle=True, batch_size=batch_size, target_size=(224, 224))

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


In [31]:
import tensorflow as tf
flatten_output = tf.keras.backend.function(pretrained_model.input, pretrained_model.get_layer('flatten').output)

In [32]:
extracted_train_features_ = []
labels = []
train_size = 9469
from IPython.display import clear_output
epochs = int(train_size/batch_size)
for i in range(epochs):
    print(((i+1)/epochs)*100)
    clear_output(wait=True)
    xs,ys = train.next()
    x = flatten_output(xs)
    extracted_train_features_.append(x)
    labels.append(ys)

100.0


In [38]:
import numpy as np
x = np.array(extracted_train_features_)
y = np.array(labels)
y = np.squeeze(y,axis=1)
x = np.squeeze(x,axis=1)
print(x.shape)
print(y.shape)

ERROR! Session/line number was not unique in database. History logging moved to new session 762
(9469, 25088)
(9469, 10)


In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
model = Sequential()
model.add(Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation
             ='softmax',input_shape=(25088,)))
model.compile(optimizer = 'adam', loss = 'squared_hinge', metrics = ['accuracy'])
model.fit(x,y, epochs=100, batch_size=1500, verbose=1, validation_split=0.2)


Train on 7575 samples, validate on 1894 samples
Epoch 1/100
7575/7575 [==============================] - 1s 97us/sample - loss: 1.3432 - acc: 0.3486 - val_loss: 1.2325 - val_acc: 0.5723
Epoch 2/100
7575/7575 [==============================] - 1s 67us/sample - loss: 1.1979 - acc: 0.6367 - val_loss: 1.1559 - val_acc: 0.7027
Epoch 3/100
7575/7575 [==============================] - 1s 72us/sample - loss: 1.1370 - acc: 0.7368 - val_loss: 1.1150 - val_acc: 0.7682
Epoch 4/100
7575/7575 [==============================] - 1s 67us/sample - loss: 1.1054 - acc: 0.7806 - val_loss: 1.0939 - val_acc: 0.7914
Epoch 5/100
7575/7575 [==============================] - 1s 72us/sample - loss: 1.0812 - acc: 0.8144 - val_loss: 1.0797 - val_acc: 0.8015
Epoch 6/100
7575/7575 [==============================] - 1s 69us/sample - loss: 1.0701 - acc: 0.8182 - val_loss: 1.0651 - val_acc: 0.8147
Epoch 7/100
7575/7575 [==============================] - 1s 71us/sample - loss: 1.0521 - acc: 0.8409 - val_loss: 1.0525 - va

7575/7575 [==============================] - 1s 74us/sample - loss: 1.0573 - acc: 0.8346 - val_loss: 1.0740 - val_acc: 0.7967
Epoch 60/100
7575/7575 [==============================] - 1s 69us/sample - loss: 1.0517 - acc: 0.8539 - val_loss: 1.0694 - val_acc: 0.8089
Epoch 61/100
7575/7575 [==============================] - 1s 74us/sample - loss: 1.0536 - acc: 0.8475 - val_loss: 1.0653 - val_acc: 0.8131
Epoch 62/100
7575/7575 [==============================] - 1s 68us/sample - loss: 1.0549 - acc: 0.8379 - val_loss: 1.0706 - val_acc: 0.7941
Epoch 63/100
7575/7575 [==============================] - 1s 71us/sample - loss: 1.0534 - acc: 0.8371 - val_loss: 1.0656 - val_acc: 0.8041
Epoch 64/100
7575/7575 [==============================] - 1s 71us/sample - loss: 1.0473 - acc: 0.8470 - val_loss: 1.0622 - val_acc: 0.8046
Epoch 65/100
7575/7575 [==============================] - 1s 70us/sample - loss: 1.0428 - acc: 0.8512 - val_loss: 1.0559 - val_acc: 0.8147
Epoch 66/100
7575/7575 [================

In [35]:
xtest = []
ytest = []
test_size = 3925
epochs = int(test_size/batch_size)
for i in range(epochs):
    print(((i+1)/epochs)*100)
    clear_output(wait=True)
    xs,xy = val.next()
    x = flatten_output(xs)
    xtest.append(x)
    ytest.append(xy)

x_test = np.array(xtest)
y_test = np.array(ytest)
x_test = np.squeeze(x_test,axis=1)
y_test = np.squeeze(y_test,axis=1)
print(x_test.shape)
print(y_test.shape)

(3925, 25088)
(3925, 10)


In [42]:
random_index = np.random.randint(low=0,high=test_size-1,size=2000)

test_results = model.evaluate(x_test[random_index], y_test[random_index], verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

2000/2000 [==============================] - 0s 82us/sample - loss: 1.0206 - acc: 0.8025
Test results - Loss: 1.020605990409851 - Accuracy: 80.25000095367432%
